# Demo notebook for Model Reader - TIEGCM Example
See Block 3 for instructions. You may run the notebook as is if you have the sample data file, but you must
change the 'filename' variable in block 3 to have the correct file path.

In [ ]:
# import block
import numpy as np
from kamodo_ccmc.readers.tiegcm_4D import MODEL, model_varnames
# Alternatively, can import reader with below logic:
# from kamodo_ccmc.flythrough.model_wrapper import Choose_Model
# module = Choose_Model('TIEGCM')   #leave input empty to retrieve a list of possible models
reader = MODEL()

In [ ]:
# Print possible variable names for given model
# Not all variables will be available in a given output file
print('Standardized variables:\n---------------------------')
for key in model_varnames.keys():
    print(model_varnames[key][0]+':', model_varnames[key][1:])
# variable name, description, variable number, coordinate type, coordinate grid, list of coordinate names, units of data
# For more information on the coordinate systems indicated below, see kamodo_ccmc.flythrough.utils.ConvertCoord in the 
#  coordinate tutorial notebook.

In [ ]:
# Print all available variables in the file selected
filename = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Data/s001.nc'  #TIEGCM file doesn't have any 3D variables
# Change the filename to reflect the path on your machine.
# This file is available at https://drive.google.com/file/d/1pHx9Q8v4vO59_RUMX-SJqYv_-dE3h-st/view?usp=sharing
# For model readers that produce more than one file per day, the input 'filename' is the naming pattern
# e.g. for the CTIPe reader: filename = 'C:/Users/rringuet/Kamodo_WinDev1/CTIPe/Data/2015-03-20'

kamodo_object = reader(filename, fulltime=False, variables_requested='all')
for key in kamodo_object.var_dict.keys():
    print(key, kamodo_object.var_dict[key])

In [ ]:
# Demonstrate various function call methods. 
# INSTRUCTIONS:
# Comment out all but one 'kamodo_object' line and rerun entire notbook to test function call variations.

# Default function call: functionalize all variables, INcluding gridded versions
# kamodo_object = reader(filename)  # (gridded_int=True is default)

# Option 1: functionalize all variables, EXcluding gridded versions
# kamodo_object = reader(filename, gridded_int=False)  

# Option 2: functionalize variables in list, INcluding gridded versions
kamodo_object = reader(filename, variables_requested=['T_n','TEC','psi_N2'])  

# Option 3: load a single variable without gridded version, and include the name of run
# kamodo_object = reader(filename, variables_requested=['T_n'], gridded_int=False, runname='Testing')  
print()

# Show functionalized variables in kamodo object
# Notes: 
# - Without the gridded function versions, there are no indications of the coordinate type (cartesian vs spherical)
#       Cartesian coordinate variables are always in R_earth units.
#       Spherical variables are typically (deg, deg, R_earth), sometimes (deg, deg, km). 
#       The next code block shows how to check this.
# - All variable names are case-sensitive
# - Requesting variables that are NOT in the list above triggers a printed message: '... name not recognized'
# - Requesting variables that ARE in the list above but not in the file triggers a printed message: '...not available'
# - If none of the variables requested are in the file, the kamodo_object variable will be empty and the remaining
#       code blocks will error.
# - Any pressure level variables that the requested variables depend on are automatically included
kamodo_object

In [ ]:
# Method to show variable coordinate dependencies for non-gridded variables (variables without 'ijk' as a subscript)
# Notice these are in spherical coordinates. 
# In this case, all variables depend on various pressure level grids (ilev, ilev1, milev), which have no units.
test_var = list(kamodo_object.variables.keys())[0]  # choose first variable in kamodo object to show functionality.
print(kamodo_object.variables[test_var]['xvec'])  # for one variable
# for key in kamodo_object.variables.keys(): print(key, kamodo_object.variables[key]['xvec'])  # for all variables

In [ ]:
# Method to show variable coordinate dependencies for a gridded variable
# Notice these are in spherical coordinates. 
# In this case, all variables depend on various pressure level grids (ilev, ilev1, milev), which have no units.
if test_var+'_ijk' not in kamodo_object.variables.keys():
    print('Gridded version of function not found.')
    grid_var = test_var
else: grid_var = test_var+'_ijk'
kamodo_object[grid_var]  #for one variable

In [ ]:
# Show how to retrieve beginning and ending time values included in model data output in various formats
print('UTC timestamp:',kamodo_object.filetimes)  # UTC timestamps
print('Python datetime object:',kamodo_object.filedate)  # python datetime object in UTC for file date (standard is midnight)
print('Datetime string:',kamodo_object.datetimes)  # string representation in 'YYYY-MM-DD HH:MM:SS' format

In [ ]:
# Plot.
kamodo_object.plot('T_n_ijk', plot_partial={'T_n_ijk': {'time': 2.5, 'ilev': 5.}})

In [ ]:
# Example of interpolation command for a regular function
kamodo_object[test_var]((2.5, -20.5, -20.5, 3.4))

In [ ]:
# Example of interpolation command for a gridded function
# This will fail if the gridded variable was not created.
# The value will be the same as the previous block's result
kamodo_object[test_var+'_ijk'](2.5, -20.5, -20.5, 3.4)